# RGB Region Cutout Generator

See notes in Obsidian for full details.

> Summary
>
> The astrometry for my *HST* images of ESO 137-001 in my [2023 ESO Paper](https://ui.adsabs.harvard.edu/abs/2023MNRAS.522..173W/abstract) for my Dissertation was off by a fraction of an arcsecond. The primary issue this creates is with Figure 1 from my ESO Paper. Ming would like me to fix Figure 1 in that paper along with any other issues that might have been introduced due to this error.

In [ ]:
# Imports
from pathlib import Path
from warnings import catch_warnings, simplefilter
import itertools
from itertools import combinations_with_replacement as cwr

# Numerical Imports
import numpy as np
from scipy import ndimage

# Astropy Collab Imports
from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs.utils import proj_plane_pixel_scales as pixscale
from astropy.nddata import Cutout2D
from astropy.coordinates import FK5, SkyCoord
from astropy import units as u
from astropy import visualization as avis
import regions

# Plotting Imports
from matplotlib import pyplot as plt
from matplotlib.patches import Circle, Ellipse, FancyArrowPatch
from tqdm import tqdm

## Notebook Setup

### Pixel Scale

In [ ]:
# Create the Equivalencies
equivs  = u.pixel_scale(0.03 * u.arcsec / u.pixel)
equivs += u.pixel_scale(101.93679918450562 * u.pixel / u.kpc)

# Add the Equivalencies to the Astropy Units for this Notebook
_ = u.set_enabled_equivalencies(equivs)

### Directories and Files

In [ ]:
# Directories
PARENT_DIR = Path('../../').resolve()
IMG_DIR = PARENT_DIR / 'Images/ProcessedImages/HST/Drizzled'
OLD_ALL_REG_DIR = PARENT_DIR / 'DoctoralWork/MAST_DATA/WaldronPipeline/DS9'
REG_DIR = OLD_ALL_REG_DIR / 'Paper1/Regions'
OUT_DIR = Path('Cutouts')
ALM_DIR = PARENT_DIR / 'DoctoralWork/ALMA'

In [ ]:
# Image File Names
IMG_FNS = {}
for filter in [275, 475, 814]:
    IMG_FNS[filter] = IMG_DIR / f'ESO137-001-F{filter:3d}W_drc.fits'

# ALMA Image
ALMA_IMG_FN = ALM_DIR / 'new_mom0.fits'

In [ ]:
# Region File Names
REG_FN_IMG = REG_DIR / 'Paper1-Fig1Regions-img.reg'
REG_FN_FK5 = REG_DIR / 'Paper1-Fig1Regions-fk5.reg'

In [ ]:
# Alma Contour Levels
ALMA_CONTOUR_LEVELS = [0.03, 0.06, 0.12]

In [ ]:
# Arrow on Cutouts
ARROW_LENGTH = (0.25 * u.kpc).to('pix').value

## Load Data

### Load Region Files

In [ ]:
# The Defined Figure 1 Regions
# ds9ImgRegs = regions.Regions.read(REG_FN_IMG)
ds9FK5Regs = regions.Regions.read(REG_FN_FK5)

# The HII Regions
hiiRegs = regions.Regions.read(OLD_ALL_REG_DIR / 'eso137_001_HII_region.reg')

# XRay Images
xrayRegs = regions.Regions.read(OLD_ALL_REG_DIR / 'sun-XRay-fk5.reg')

### Load ALMA Image

In [ ]:
alma = {}
with fits.open(ALMA_IMG_FN) as hduList, catch_warnings():
    simplefilter('ignore')

    # Get Image and Header
    alma['img'] = hduList[0].data
    alma['hdr'] = hduList[0].header

    # Remove Non-Existent WCS Info
    for i, key in itertools.product([3, 4], ['CTYPE', 'CRVAL', 'CDELT', 'CRPIX', 'CUNIT']):
        alma['hdr'].remove(key + str(i))

    # Remove Other Transform Stuff
    for i, j in cwr(range(1, 5), 2):
        if not (i in [1, 2] and j in [1, 2]):
            alma['hdr'].remove(f'PC0{i}_0{j}')
            alma['hdr'].remove(f'PC0{j}_0{i}', ignore_missing=True)

    # Setup the WCS
    alma['wcs'] = WCS(alma['hdr'])

### Load *HST* Images

In [ ]:
# Init Images
imgs = {}

# Load Loop
for filter, fn in IMG_FNS.items():

    # Get the Image
    with fits.open(fn) as hduList:

        imgs[filter] = hduList['SCI'].data
        if filter == 475:
            mainWCS = WCS(hduList['SCI'].header)

### Filter Images

In [ ]:
# Filter
fImgs = {
    275: ndimage.gaussian_filter(imgs[275], sigma=1),
    475: ndimage.gaussian_filter(imgs[475], sigma=1),
    814: ndimage.gaussian_filter(imgs[814], sigma=1),
}

### Normalize Images

In [ ]:
# Close
plt.close('all')

# Init
nImgs = {}

# F275W
norm = avis.ManualInterval(     1e-6, 0.0498653) + avis.LogStretch(30)# + avis.AsymmetricPercentileInterval(5, 100)
nImgs[275] = norm(fImgs[275])

# F475W
norm = avis.ManualInterval(0.0050/100, 0.602034) + avis.LogStretch(30)
nImgs[475] = norm(fImgs[475])

# F814W
norm = avis.ManualInterval(0.0055/100, 0.729401) + avis.LogStretch(30)
nImgs[814] = norm(fImgs[814])

# Get the Composite Image
r, g, b = nImgs[814], nImgs[475], nImgs[275]
cImg    = np.dstack((r, g, b))
cImg40  = np.dstack((r, g, g))

## Plot

In [ ]:
# Region File Name Format
outFmt = OUT_DIR / 'Region{:02d}.png'

# Get the Axes
fig = plt.figure(figsize=(97/72, 97/72), dpi=300)
ax = fig.add_subplot(1, 1, 1, projection=mainWCS)
fig.add_axes(ax)
_ = ax.set_title('')
_ = ax.axis('off')
_ = ax.set_position((0, 0, 1, 1))

# Plot the Image
axImg = ax.imshow(cImg, zorder=0)

# Plot the HII Regions
rad  = (0.4 * u.arcsec).to('pix').value
for reg in hiiRegs:

    # Get the Coords
    xy = np.array(mainWCS.world_to_pixel(reg.center))

    # Plot the Circle
    cir = Circle(
        xy, rad, linestyle=(0, (3, 2)), linewidth=1,
        ec='w', fill=False, zorder=3
    )

    # Add
    ax.add_patch(cir)

# Plot the Chandra XRay Sources
rad  = (0.5 * u.arcsec).to('pix').value
for reg in xrayRegs:

    # Get the Coords
    xy = np.array(mainWCS.world_to_pixel(reg.center))

    # Plot the Circle
    cir = Circle(
        xy, rad, linestyle=(0, (1, 1)), linewidth=1,
        ec=(0, 1, 1), fill=False, zorder=2
    )

    # Add
    ax.add_patch(cir)

# Set the Cutout Limits
for i, reg in tqdm(enumerate(ds9FK5Regs)):

    # Plot the ALt Image for Region 40
    if i+1 == 40:
        axImg.set_visible(False)
        _ = ax.imshow(cImg40, zorder=0, transform=ax.get_transform(mainWCS))

    # Get the Cutout Coordinates
    w = reg.width.to(u.deg).value
    h = reg.height.to(u.deg).value

    # Add ALMA CO Contours
    alma['cut'] = Cutout2D(
        alma['img'], reg.center, (h, w)*u.deg + 1*u.arcsec, wcs=alma['wcs']
    )
    if i+1 !=24:
        alma['fil'] = ndimage.gaussian_filter(alma['cut'].data, sigma=0.75)
    else:
        alma['fil'] = alma['cut'].data
    if np.any(np.histogram(alma['fil'], ALMA_CONTOUR_LEVELS)[1:-1]):
        ax.contour(
            alma['fil'],
            levels=ALMA_CONTOUR_LEVELS,
            colors='xkcd:magenta', linewidths=1, zorder=1,
            transform=ax.get_transform(alma['cut'].wcs)
        )

    # Convert to Image
    x, y = mainWCS.world_to_pixel(reg.center)
    w   /= pixscale(mainWCS)[0]
    h   /= pixscale(mainWCS)[0]
    xMin, xMax = x-w/2, x+w/2
    yMin, yMax = y-h/2, y+h/2

    # Add the Scale Arrow
    arr = FancyArrowPatch(
        (xMin+w/10, yMin+w/10),
        (xMin+w/10+ARROW_LENGTH, yMin+w/10),
        ec='w', arrowstyle='|-|', capstyle='butt', lw=0.75, zorder=4
    )
    ax.add_patch(arr)

    # Set the Limits
    _ = ax.set_xlim(xMin, xMax)
    _ = ax.set_ylim(yMin, yMax)

    # Save the Cutout
    fig.savefig(str(outFmt).format(i+1))